# Project Part 3: Final Presentation

### CS-GY 6083 Principals of Database Systems - Spring 2024

* **Author**: [Nicola Maiorana]
* **Date**: [2024-04-14]
* **Email**: [nam10102@nyu.edu]
* **Class**: [CS-GY 6083]
---

## Overview of Presentation
- Project Overview
- IDE And Presentation Tools
- Code
    - ER Diagram
    - Tables DDL (8 Tables)
    - Views DDL (3 Views)
    - Procedures DDL (2 Procedures)
    - Functions DDL (2 Functions)
    - Triggers DDL (2 Triggers)
    - Business Objects (8 Classes)
- Forms
    - Table Inserts
    - Table Deletes
    - Selects
- Reports
    - Total Sales
    - Total Units Sold
---

## Project Overview
- MySQL Database
- Python
- 2-Tier Architecture
- Record Album Information:
    - Album Name, Recording Date, Artists, Record Label, Genre, Sales Information
    - Supporting Tables:
        - Artist, Genre, Record Label, Album, Track, Sales Tables
- Business Objects:
    - RecordArtist, RecordGenre, RecordLabel, RecordAlbum, RecordTrack, RecordSale, GroupMember, MembersToArtists
    - CRUD Operations:
        - Create, Read, Update, Delete
    - Helper Operations:
        - Read All, Read By Id, Read By Name, Add Member, Remove Member, ...

- Forms Demonstration:
    - Record Album Manager
    - Record Tracks Manager

---

In [5]:
import tools.db_utils as dbu
import tools.mermaid_diagrams as md

### Import required modules

- tools.db_utils: Contains the database connection and query functions
- tools.mermaid_diagrams: Allows for the creation of Mermaid diagrams

## Code

### ER Diagram
- Normalization
- Integrity Enforcement Used (Where/Why)
- Isolation Level Used (Where/Why)

In [4]:
md.mm(md.er_diargram)

---
## Tables DDL
- Record Artists
- Group Members
- Members To Artists
- Record Genres
- Record Labels
- Record Albums
- Record Tracks
- Record Sales
```sql
CREATE TABLE RECORD_ARTISTS (
    artist_id int NOT NULL AUTO_INCREMENT,
    artist_name varchar(40),
    PRIMARY KEY (artist_id)
);

CREATE TABLE GROUP_MEMBERS (
    member_id int NOT NULL AUTO_INCREMENT,
    member_name varchar(40),
    member_country varchar(40),
    member_birthdate date,
    PRIMARY KEY (member_id)
);

CREATE TABLE MEMBERS_TO_ARTISTS (
    members_to_artists_id int NOT NULL AUTO_INCREMENT,
    member_id int,
    artist_id int,
    member_from_date date,
    member_to_date date,
    PRIMARY KEY (members_to_artists_id),
    FOREIGN KEY (member_id) REFERENCES GROUP_MEMBERS(member_id),
    FOREIGN KEY (artist_id) REFERENCES RECORD_ARTISTS(artist_id)
);

CREATE TABLE RECORD_GENRES (
    genre_id int NOT NULL AUTO_INCREMENT,
    genre_name varchar(40),
    genre_description varchar(255),
    PRIMARY KEY (genre_id),
    UNIQUE (genre_name)
);

CREATE TABLE RECORD_LABELS (
    record_label_id int NOT NULL AUTO_INCREMENT,
    record_label_name varchar(50),
    PRIMARY KEY (record_label_id),
    UNIQUE (record_label_name)
);

CREATE TABLE RECORD_ALBUMS (
    album_id int NOT NULL AUTO_INCREMENT,
    album_name varchar(50),
    release_date date,
    artist_id int,
    genre_id int,
    record_label_id int,
    PRIMARY KEY (album_id),
    FOREIGN KEY (artist_id) REFERENCES RECORD_ARTISTS(artist_id),
    FOREIGN KEY (genre_id) REFERENCES RECORD_GENRES(genre_id),
    FOREIGN KEY (record_label_id) REFERENCES RECORD_LABELS(record_label_id),
    UNIQUE (album_name, release_date)
);

CREATE TABLE RECORD_TRACKS (
    track_id int NOT NULL AUTO_INCREMENT,
    album_id int,
    track_name varchar(40),
    track_number int,
    genre_id int,
    PRIMARY KEY (track_id),
    FOREIGN KEY (album_id) REFERENCES RECORD_ALBUMS(album_id),
    FOREIGN KEY (genre_id) REFERENCES RECORD_GENRES(genre_id),
    UNIQUE (album_id, track_number)
);

CREATE TABLE RECORD_SALES (
    sale_id int NOT NULL AUTO_INCREMENT,
    album_id int,
    sale_date date,
    sale_quantity int,
    unit_sale_price decimal,
    PRIMARY KEY (sale_id),
    FOREIGN KEY (album_id) REFERENCES RECORD_ALBUMS(album_id)
);
```

### Database Utility Functions

- Query DB and return a pandas DataFrame

In [7]:
import tools.db_utils as dbu
pandas_index_settings = ['album_id']
display(dbu.query_to_df(('select '
                            'rec.album_id, '
                            'rec.album_name, '
                            'ra.artist_name, '
                            'rg.genre_name, '
                            'rg.genre_description, '
                            'rl.record_label_name '
                            'from record_albums rec '
                            'JOIN record_artists ra ON ra.artist_id = rec.artist_id '
                            'JOIN record_genres rg ON rg.genre_id = rec.genre_id '
                            'JOIN record_labels rl ON rl.record_label_id = rec.record_label_id'), pandas_index_settings))

,album_name,artist_name,genre_name,genre_description,record_label_name
album_id,,,,,
1,The Dark Side of the Moon,Pink Floyd,Rock,Rock music,Harvest Capitol Records
2,Boston,Boston,Rock,Rock music,Epic Records
3,Watermark,Enya,New Age,New Age music,Warner Bros. Records


___
### Views DDL
- Album Information
- Album Information Details
- Band Members

#### Album Information View
```sql
CREATE VIEW album_information AS
    SELECT
        RECORD_ALBUMS.album_name,
        RECORD_ARTISTS.artist_name,
        RECORD_GENRES.genre_name,
        RECORD_LABELS.record_label_name,
        RECORD_ALBUMS.release_date
    FROM
        RECORD_ALBUMS
    JOIN RECORD_ARTISTS ON RECORD_ALBUMS.artist_id = RECORD_ARTISTS.artist_id
    JOIN RECORD_LABELS ON RECORD_ALBUMS.record_label_id = RECORD_LABELS.record_label_id
    JOIN RECORD_GENRES ON RECORD_ALBUMS.genre_id = RECORD_GENRES.genre_id;
```

In [7]:
import tools.db_utils as dbu
pandas_index_settings = ['album_name']
display(dbu.query_to_df('select * from album_information order by release_date', pandas_index_settings))

,artist_name,genre_name,record_label_name,release_date
album_name,,,,
The Dark Side of the Moon,Pink Floyd,Rock,Harvest Capitol Records,1973-03-01
Boston,Boston,Rock,Epic Records,1976-08-25
Watermark,Enya,New Age,Warner Bros. Records,1988-09-19


#### Album Information Details View
```sql
CREATE VIEW album_information_details AS
    SELECT
        RECORD_ALBUMS.album_name,
        RECORD_ARTISTS.artist_name,
        record_genres.genre_name,
        RECORD_LABELS.record_label_name,
        RECORD_TRACKS.track_number,
        RECORD_TRACKS.track_name,
        RECORD_ALBUMS.release_date
    FROM
        RECORD_ALBUMS
    JOIN RECORD_ARTISTS ON RECORD_ALBUMS.artist_id = RECORD_ARTISTS.artist_id
    JOIN RECORD_LABELS ON RECORD_ALBUMS.record_label_id = RECORD_LABELS.record_label_id
    JOIN RECORD_TRACKS ON RECORD_ALBUMS.album_id = RECORD_TRACKS.album_id
    JOIN RECORD_GENRES ON RECORD_TRACKS.genre_id = RECORD_GENRES.genre_id
ORDER BY
    RECORD_ALBUMS.album_name, RECORD_TRACKS.track_number;
```

In [8]:
import tools.db_utils as dbu
pandas_index_settings = ['album_name', 'release_date', 'artist_name', 'record_label_name', 'track_number']
display(dbu.query_to_df('select * from album_information_details order by release_date', pandas_index_settings))

genre_name  \
album_name                release_date artist_name record_label_name       track_number              
The Dark Side of the Moon 1973-03-01   Pink Floyd  Harvest Capitol Records 1                  Rock   
                                                                           2                  Rock   
                                                                           3                  Rock   
                                                                           4                  Rock   
                                                                           5                  Rock   
                                                                           6                  Rock   
                                                                           7                  Rock   
                                                                           8                  Rock   
                                                                           9                  Rock   
                                                                           10                 Rock   
Boston                    1976-08-25   Boston      Epic Records            1                  Rock   
                                                                           2                  Rock   
                                                                           3                  Rock   
                                                                           4                  Rock   
                                                                           5                  Rock   
                                                                           6                  Rock   
                                                                           7                  Rock   
                                                                           8                  Rock   
Watermark                 1988-09-19   Enya        Warner Bros. Records    1               New Age   
                                                                           2               New Age   
                                                                           3               New Age   
                                                                           4               New Age   
                                                                           5               New Age   
                                                                           6               New Age   
                                                                           7               New Age   
                                                                           8               New Age   
                                                                           9               New Age   
                                                                           10              New Age   
                                                                           11              New Age   

                                                                                                           track_name  
album_name                release_date artist_name record_label_name       track_number                                
The Dark Side of the Moon 1973-03-01   Pink Floyd  Harvest Capitol Records 1                              Speak to Me  
                                                                           2                     Breathe (In the Air)  
                                                                           3                               On the Run  
                                                                           4                                     Time  
                                                                           5                 The Great Gig in the Sky  
                                                                           6         

#### Band Members View
```sql
CREATE VIEW band_members AS
    SELECT
        RECORD_ARTISTS.artist_name,
        GROUP_MEMBERS.member_name,
        GROUP_MEMBERS.member_country,
        MEMBERS_TO_ARTISTS.member_from_date,
        MEMBERS_TO_ARTISTS.member_to_date
    FROM
        RECORD_ARTISTS
    JOIN MEMBERS_TO_ARTISTS ON RECORD_ARTISTS.artist_id = MEMBERS_TO_ARTISTS.artist_id
    JOIN GROUP_MEMBERS ON MEMBERS_TO_ARTISTS.member_id = GROUP_MEMBERS.member_id;
```

In [5]:
import tools.db_utils as dbu
pandas_index_settings = ['artist_name', 'member_name']
display(dbu.query_to_df('select * from band_members order by artist_name, member_name', pandas_index_settings))

member_country member_from_date member_to_date
artist_name member_name                                                  
Boston      Barry Goudreau            USA       1975-01-01     1981-01-01
            Brad Delp                 USA       1975-01-01     2007-01-01
            Fran Sheehan              USA       1975-01-01     1986-01-01
            Sib Hashian               USA       1975-01-01     1986-01-01
            Tom Scholz                USA       1975-01-01     1989-01-01
Enya        Enya                  Ireland       1982-01-01     2020-01-01
Pink Floyd  David Gilmour         England       1968-01-01     1995-01-01
            Nick Mason            England       1965-01-01     1995-01-01
            Richard Wright        England       1965-01-01     1980-01-01
            Roger Waters          England       1965-01-01     1985-01-01
            Syd Barrett           England       1965-01-01     1968-01-01

___
### Procedures DDL
- Count Record Sales
- Total Record Sales


### Procedure to count the number of sales for a given album
```
DELIMITER //
CREATE PROCEDURE count_record_sales(IN album_id INT, OUT sales_count INT)
BEGIN
	SET @album_id = album_id;
    SELECT 
        COUNT(sale_id) INTO sales_count
    FROM RECORD_SALES
    WHERE RECORD_SALES.album_id = @album_id;

END //
DELIMITER ;
```


In [14]:
import tools.db_utils as dbu
from business_objects.record_album_sql import RecordAlbum
from business_objects.record_sales_sql import RecordSales
with dbu.get_connector() as conn:
    with conn.cursor() as cur:
        record_album = RecordAlbum.read_by_name("Boston")
        all_sales = RecordSales.read_by_album_id(record_album.album_id)
        print(f'Number of sales for album {record_album.album_name} using select statement    : {len(all_sales)}')
        result = cur.callproc('count_record_sales', [record_album.album_id, 0])[1]
        print(f'Number of sales for album {record_album.album_name} using the stored procedure: {result}')

Number of sales for album Boston using select statement    : 12
Number of sales for album Boston using the stored procedure: 12


### Procedure to get the total sales for all albums
```sql
DELIMITER //
CREATE PROCEDURE total_record_sales(IN album_id INT, OUT total_sales NUMERIC)
BEGIN
	SET @album_id = album_id;
    SELECT 
        sum(sale_quantity * unit_sale_price) INTO total_sales
    FROM RECORD_SALES
    WHERE RECORD_SALES.album_id = @album_id;

END //
DELIMITER ;
```

In [18]:
import tools.db_utils as dbu
from business_objects.record_album_sql import RecordAlbum
with dbu.get_connector() as conn:
    with conn.cursor() as cur:
        record_album = RecordAlbum.read_by_name("Boston")
        total_sales_query = dbu.query_to_df(f"select sum(sale_quantity * unit_sale_price) as total_sales from record_sales where album_id = {record_album.album_id}")["total_sales"][0]
        print(f'Total sales for album {record_album.album_name} using select statement    : {total_sales_query}')
        result = float(cur.callproc('total_record_sales', (record_album.album_id, 0))[1])
        print(f'Total sales for album {record_album.album_name} using the stored procedure: {result}')

Total sales for album Boston using select statement    : 120000000.0
Total sales for album Boston using the stored procedure: 120000000.0


___
### Function DLL
- Album Summary Function to return a summary for an album to include the artist, producer, release date and tracks
```sql
DELIMITER //
CREATE FUNCTION album_summary (album_id int) RETURNS varchar(600)
	READS SQL DATA
BEGIN
	DECLARE done int default false;
    DECLARE track_summary varchar(400);
	DECLARE album_summary varchar(200);
    DECLARE response_summary varchar(600);
	DECLARE result_cursor CURSOR FOR
		SELECT
			concat(" ", t.track_number, "-", t.track_name) AS track_summary
			FROM record_tracks t
			WHERE t.album_id = album_id
			ORDER BY t.track_number;
	DECLARE CONTINUE HANDLER FOR not found SET done = true;
	SET album_summary = "";
    SET response_summary = "";
	OPEN result_cursor;
	read_loop: LOOP
		FETCH result_cursor INTO track_summary;
        IF done THEN
			LEAVE read_loop;
		END IF;
        SET response_summary = CONCAT(response_summary, track_summary, "\n ");
    END LOOP;
    CLOSE result_cursor;
	SELECT
		concat(
			r.album_name, "\n ",
			"Perfomed by: ", a.artist_name, "\n ",
            "Produced by: ", l.record_label_name, "\n ",
			"Released   : ", r.release_date, "\n "
            ) INTO album_summary
		FROM record_albums r, record_artists a, record_labels l
		WHERE r.album_id = album_id and a.artist_id = r.artist_id and l.record_label_id = r.record_label_id;
	SET response_summary = CONCAT(album_summary, response_summary);
    RETURN response_summary;
END //
DELIMITER ;
```

In [19]:
import tools.db_utils as dbu
from business_objects.record_album_sql import RecordAlbum
with dbu.get_connector() as conn:
    with conn.cursor() as cur:
        record_album = RecordAlbum.read_by_name("Boston")
        cur.execute(f'select album_summary(%s)', (record_album.album_id,))
        result = cur.fetchone()[0]
        print(f'Album summary:\n {result}')

Album summary:
 Boston
 Perfomed by: Boston
 Produced by: Epic Records
 Released   : 1976-08-25
  1-More Than a Feeling
  2-Peace of Mind
  3-Foreplay/Long Time
  4-Rock & Roll Band
  5-Smokin
  6-Hitch a Ride
  7-Something About You
  8-Let Me Take You Home Tonight
 


### Function to show all the members of a band over time

```sql
# Function to summarize an band members for an artist
DROP FUNCTION IF EXISTS band_summary;
DELIMITER //
CREATE FUNCTION band_summary (artist_id int) RETURNS varchar(600)
	READS SQL DATA
BEGIN
	DECLARE done int default false;
    DECLARE member_summary varchar(400);
	DECLARE band_summary varchar(200);
    DECLARE response_summary varchar(600);
	DECLARE result_cursor CURSOR FOR
		SELECT
			concat(" ", group_members.member_name,
				   " (", group_members.member_country, ")", 
				   " from: ",  members_to_artists.member_from_date, 
				   " till: ", members_to_artists.member_to_date) AS member_summary
			FROM
				record_artists
			JOIN members_to_artists ON record_artists.artist_id = members_to_artists.artist_id
			JOIN group_members ON members_to_artists.member_id = group_members.member_id
			WHERE record_artists.artist_id = artist_id;
	DECLARE CONTINUE HANDLER FOR not found SET done = true;
	SET band_summary = "";
    SET response_summary = "";
	OPEN result_cursor;
	read_loop: LOOP
		FETCH result_cursor INTO member_summary;
        IF done THEN
			LEAVE read_loop;
		END IF;
        SET response_summary = CONCAT(response_summary, member_summary, "\n ");
    END LOOP;
    CLOSE result_cursor;
	SELECT
		concat(a.artist_name, " - Band Members: \n ") INTO band_summary
		FROM record_artists a
		WHERE a.artist_id = artist_id;
	SET response_summary = CONCAT(band_summary, response_summary);
    RETURN response_summary;
END //
DELIMITER ;
```

In [21]:
import tools.db_utils as dbu
from business_objects.record_artists_sql import RecordArtist
with dbu.get_connector() as conn:
    with conn.cursor() as cur:
        artist_id = 1
        record_artist = RecordArtist.read_by_name("Pink Floyd")
        cur.execute(f'select band_summary(%s)', (record_artist.artist_id,))
        result = cur.fetchone()[0]
        print(f'Summary for {record_artist.artist_name}:\n{result}')

Summary for Pink Floyd:
Pink Floyd - Band Members: 
  Syd Barrett (England) from: 1965-01-01 till: 1968-01-01
  Roger Waters (England) from: 1965-01-01 till: 1985-01-01
  Richard Wright (England) from: 1965-01-01 till: 1980-01-01
  Nick Mason (England) from: 1965-01-01 till: 1995-01-01
  David Gilmour (England) from: 1968-01-01 till: 1995-01-01
 


___
### Trigger DDL
- Cleanup Artists
- Cleanup Tracks

### Trigger to cleanup members_to_artists and albums when an artist is deleted
```sql
DROP TRIGGER IF EXISTS cleanup_artists;

DELIMITER //
CREATE TRIGGER cleanup_artists BEFORE DELETE ON record_artists
FOR EACH ROW
BEGIN
    DELETE FROM members_to_artists m WHERE m.artist_id = OLD.artist_id;
    UPDATE record_albums r set artist_id = null WHERE r.artist_id = OLD.artist_id;
END //
DELIMITER ;
```

In [28]:
import tools.db_utils as dbu
from business_objects.record_artists_sql import RecordArtist
from business_objects.members_to_artists_sql import MembersToArtists
pandas_index_settings = ['artist_name', 'member_name']

with dbu.get_connector() as conn:
    with conn.cursor() as cur:
        record_artist = RecordArtist.read_by_name("Boston")
        if record_artist is None:
            print(f'Artist Boston not found. Init data...')
            
        else:
            all_members = MembersToArtists.read_members(record_artist.artist_id)
            print(f'Number of members for artist {record_artist.artist_name} using business object    : {len(all_members)}')
            print(f'Deleting artist {record_artist.artist_name}...')
            cur.execute('delete from record_artists where artist_name = "Boston"')
            conn.commit()
            all_members = MembersToArtists.read_members(record_artist.artist_id)
            print(f'Number of members for artist {record_artist.artist_name} using business object    : {len(all_members)}')

Number of members for artist Boston using business object    : 5
Deleting artist Boston...
Number of members for artist Boston using business object    : 0


### Trigger to cleanup tracks and sales when an album is deleted
```sql
DROP TRIGGER IF EXISTS cleanup_tracks;

DELIMITER //
CREATE TRIGGER cleanup_tracks BEFORE DELETE ON record_albums
FOR EACH ROW
BEGIN
    DELETE FROM record_tracks t WHERE t.album_id = OLD.album_id;
    DELETE FROM record_sales s WHERE s.album_id = OLD.album_id;
END //
DELIMITER ;
```

In [31]:
import tools.db_utils as dbu
from business_objects.record_album_sql import RecordAlbum
from business_objects.record_tracks_sql import RecordTrack
from business_objects.record_sales_sql import RecordSales

pandas_index_settings = ['album_name', 'release_date', 'artist_name', 'record_label_name', 'track_number']
with dbu.get_connector() as conn:
    with conn.cursor() as cur:
        record_album = RecordAlbum.read_by_name("Boston")
        if record_album is None:
            print(f'Album Boston not found. Init data...')
            
        else:
            record_tracks = RecordTrack.read_by_album_id(record_album.album_id)
            record_sales = RecordSales.read_by_album_id(record_album.album_id)
            print(f'Number of tracks for album {record_album.album_name} using business object    : {len(record_tracks)}')
            print(f'Number of sales for album {record_album.album_name} using business object     : {len(record_sales)}')
            print(f'Deleting album {record_album.album_name}...')
            cur.execute('delete from record_albums where album_name = "Boston"')
            conn.commit()
            record_tracks = RecordTrack.read_by_album_id(record_album.album_id)
            record_sales = RecordSales.read_by_album_id(record_album.album_id)
            print(f'Number of tracks for album {record_album.album_name} using business object    : {len(record_tracks)}')
            print(f'Number of sales for album {record_album.album_name} using business object     : {len(record_sales)}')

Number of tracks for album Boston using business object    : 8
Number of sales for album Boston using business object     : 12
Deleting album Boston...
Number of tracks for album Boston using business object    : 0
Number of sales for album Boston using business object     : 0


---
## Reports

- Album Total Sales
- Album Annual Sales

In [14]:
import tools.db_utils as dbu
pandas_index_settings = ['Album Name']
print(f'Total Sales')
display(dbu.query_to_df(('SELECT '
                            'ra.album_name as "Album Name", '
                            'SUM(rs.sale_quantity) as "Units Sold", '
                            'SUM(rs.sale_quantity * rs.unit_sale_price) as "Total Sales" '
                        'FROM record_sales rs '
                        'JOIN record_albums ra ON ra.album_id = rs.album_id '
                        'GROUP BY ra.album_name '
                        'ORDER BY ra.album_name'), pandas_index_settings))

pandas_index_settings = ['Album Name', 'Year']
print(f'Annual Sales')
display(dbu.query_to_df(('SELECT '
                            'ra.album_name as "Album Name", '
                            'YEAR(rs.sale_date) as "Year", '
                            'SUM(rs.sale_quantity) as "Units Sold", '
                            'SUM(rs.sale_quantity * rs.unit_sale_price) as "Sales" '
                        'FROM record_sales rs '
                        'JOIN record_albums ra ON ra.album_id = rs.album_id '
                        'GROUP BY YEAR(rs.sale_date), rs.album_id '
                        'ORDER BY ra.album_name'), pandas_index_settings))

Total Sales


,Units Sold,Total Sales
Album Name,,
Boston,1168200.0,11150000.0
The Dark Side of the Moon,3610870.0,38946350.0
Watermark,1395789.0,18183945.0


Annual Sales


Units Sold       Sales
Album Name                Year                        
Boston                    1976    120000.0   1200000.0
                          1977    130000.0   1300000.0
                          1978    200000.0   2000000.0
                          1979    250000.0   2500000.0
                          1980    175000.0   1750000.0
                          1981    120000.0   1200000.0
                          1982     75000.0    525000.0
                          1983     60000.0    420000.0
                          1984     20000.0    140000.0
                          1985     12000.0     84000.0
                          1986      5000.0     25000.0
                          1987      1200.0      6000.0
The Dark Side of the Moon 1973    750000.0   7500000.0
                          1974   1200000.0  14400000.0
                          1975    500000.0   6000000.0
                          1976    340000.0   3400000.0
                          1977    276000.0   2760000.0
                          1978    170000.0   1700000.0
                          1979    125000.0   1250000.0
                          1980     96000.0    768000.0
                          1981     70000.0    560000.0
                          1982     40000.0    320000.0
                          1983     23000.0    184000.0
                          1984     12000.0     60000.0
                          1985      4670.0     23350.0
                          1986      3400.0     17000.0
                          1987       800.0      4000.0
Watermark                 1989    300000.0   4500000.0
                          1990    240000.0   3600000.0
                          1991    200000.0   3000000.0
                          1992    189000.0   2835000.0
                          1993    150000.0   1500000.0
                          1994    125000.0   1250000.0
                          1995     80000.0    640000.0
                          1996     55000.0    440000.0
                          1997     30000.0    240000.0
                          1998     15000.0    120000.0
                          1999      8000.0     40000.0
                          2000      2300.0     11500.0
                          2001      1489.0      7445.0

___
## Forms Demonstration
- Record Album Manager
- Record Tracks Manager

In [5]:
from ui_objects.record_album_manager_ui import RecordAlbumUI

RecordAlbumUI()


### Table Inserts

In [6]:
import tools.db_utils as dbu
from business_objects.record_genres_sql import RecordGenre

# Setup
test_genre_name = 'TEST Genre'
RecordGenre.delete_by_name(test_genre_name)
# Insert
new_record = RecordGenre.create(genre_name=test_genre_name, genre_description='A new genre')
display(dbu.query_to_df(f"select * from record_genres where genre_name = '{test_genre_name}'",  'genre_id'))
# Cleanup
RecordGenre.delete_by_name(test_genre_name)

,genre_name,genre_description
genre_id,,
39,TEST Genre,A new genre


### Table Reads

In [4]:
from business_objects.record_album_sql import RecordAlbum
print(f'Read all albumns')
for album in RecordAlbum.read_all()[:5]:
    print(album)
    
print(f'\n\nRead by id: 5')
print(RecordAlbum.read(1))
print(f'\n\nRead by name: The Dark Side of the Moon')
print(RecordAlbum.read_by_name('The Dark Side of the Moon'))

Read all albumns
RecordAlbum(album_id=1, album_name='The Dark Side of the Moon', release_date=datetime.date(2024, 4, 21), artist_id=1, genre_id=1, record_label_id=2)
RecordAlbum(album_id=2, album_name='Boston', release_date=datetime.date(1976, 8, 25), artist_id=2, genre_id=1, record_label_id=1)
RecordAlbum(album_id=3, album_name='Watermark', release_date=datetime.date(1988, 9, 19), artist_id=3, genre_id=17, record_label_id=6)
RecordAlbum(album_id=7, album_name='Test', release_date=datetime.date(2024, 4, 21), artist_id=6, genre_id=4, record_label_id=4)


Read by id: 5
RecordAlbum(album_id=1, album_name='The Dark Side of the Moon', release_date=datetime.date(2024, 4, 21), artist_id=1, genre_id=1, record_label_id=2)


Read by name: The Dark Side of the Moon
RecordAlbum(album_id=1, album_name='The Dark Side of the Moon', release_date=datetime.date(2024, 4, 21), artist_id=1, genre_id=1, record_label_id=2)


### Table Updates

In [18]:
from business_objects.record_genres_sql import RecordGenre

# Setup
test_genre_name = 'TEST Genre'
updated_genre_name = 'Updated Genre'
RecordGenre.delete_by_name(test_genre_name)
RecordGenre.delete_by_name(updated_genre_name)
new_record = RecordGenre.create(genre_name=test_genre_name, genre_description='A new genre')
display(dbu.query_to_df(f"select * from record_genres where genre_name = '{test_genre_name}'",  'genre_id'))
# Update
new_record.genre_name = updated_genre_name
new_record.genre_description = 'An updated genre'
print(f'Record to update: {new_record}')
new_record.update()
display(dbu.query_to_df(f"select * from record_genres where genre_name = '{updated_genre_name}'",  'genre_id'))
# Cleanup
RecordGenre.delete_by_name(test_genre_name)
RecordGenre.delete_by_name(updated_genre_name)

,genre_name,genre_description
genre_id,,
50,TEST Genre,A new genre


Record to update: RecordGenre(genre_id=50, genre_name='Updated Genre', genre_description='An updated genre')


,genre_name,genre_description
genre_id,,
50,Updated Genre,An updated genre


### Table Deletes

In [19]:
from business_objects.record_genres_sql import RecordGenre

# Setup
test_genre_name = 'TEST Genre'
RecordGenre.delete_by_name(test_genre_name)
new_record = RecordGenre.create(genre_name=test_genre_name, genre_description='A new genre')
print(f'Created record: {new_record.genre_id}')
display(dbu.query_to_df(f"select * from record_genres where genre_name = '{test_genre_name}'",  'genre_id'))
# Delete by id
new_record.delete()
print(f'After delete by id: {new_record.genre_id}')
display(dbu.query_to_df(f"select * from record_genres where genre_id = '{new_record.genre_id}'",  'genre_id'))
# Delete by name
new_record = RecordGenre.create(genre_name=test_genre_name, genre_description='A new genre')
print(f'Created record: {new_record.genre_id}')
display(dbu.query_to_df(f"select * from record_genres where genre_name = '{test_genre_name}'",  'genre_id'))
RecordGenre.delete_by_name(new_record.genre_name)
print(f'After delete by name: {new_record.genre_name}')
display(dbu.query_to_df(f"select * from record_genres where genre_name = '{test_genre_name}'",  'genre_id'))

Created record: 51


,genre_name,genre_description
genre_id,,
51,TEST Genre,A new genre


After delete by id: 51


,genre_name,genre_description
genre_id,,


Created record: 52


,genre_name,genre_description
genre_id,,
52,TEST Genre,A new genre


After delete by name: TEST Genre


,genre_name,genre_description
genre_id,,
